# Web Scraping

In [1]:
import requests
import lxml
import bs4
import pandas as pd
import time

In [2]:
uRl = requests.get("http://books.toscrape.com/catalogue/category/books/travel_2/index.html") #saving the link to the var uRl

In [3]:
soup = bs4.BeautifulSoup(uRl.text)

In [4]:
side_cat = soup.select('.side_categories') #scrapes the menubar
len(side_cat)

1

In [5]:
list_cat = side_cat[0].select('ul') #gets the list of values in the side menubar


In [6]:
topics = list_cat[0].select('li')[0].select('a')
len(topics)

51

In [7]:
topic_list = []                                     #creating list of text values from the topics
for i in range(len(topics)):
    topic_list.append(topics[i].getText())



In [8]:

def Split_books(Books_list):
    topics = []
    for i in Books_list:
        i = i.strip().split()
    #    print(len(i))
        if len(i)>1:
            i = '-'.join(i)
        topics.append(''.join(i).lower())
    return topics
        

In [9]:
Books_list = Split_books(topic_list)

In [10]:
Books_list                                        #defined list

['books',
 'travel',
 'mystery',
 'historical-fiction',
 'sequential-art',
 'classics',
 'philosophy',
 'romance',
 'womens-fiction',
 'fiction',
 'childrens',
 'religion',
 'nonfiction',
 'music',
 'default',
 'science-fiction',
 'sports-and-games',
 'add-a-comment',
 'fantasy',
 'new-adult',
 'young-adult',
 'science',
 'poetry',
 'paranormal',
 'art',
 'psychology',
 'autobiography',
 'parenting',
 'adult-fiction',
 'humor',
 'horror',
 'history',
 'food-and-drink',
 'christian-fiction',
 'business',
 'biography',
 'thriller',
 'contemporary',
 'spirituality',
 'academic',
 'self-help',
 'historical',
 'christian',
 'suspense',
 'short-stories',
 'novels',
 'health',
 'politics',
 'cultural',
 'erotica',
 'crime']

In [11]:
base_url = "http://books.toscrape.com/catalogue/category/books/{}_{}/index.html"

In [12]:
request = requests.get(base_url.format(Books_list[1],2))
type(request)

requests.models.Response

In [13]:
soup = bs4.BeautifulSoup(request.text)

In [14]:
class Book_Scrap:
    def __init__(self):
        
        self.df = pd.DataFrame(columns=['Category','Names'])
    def books_name(self,soup):
        
        self.soup = soup
        Books = self.soup.select('.product_pod')
        
        Books_list = []
        for book in Books:
            book = book.select('a')
           
            Books_list.append(book[1].getText())
        return Books_list
      
    
    def urls(self,base_url,cat,index):
        self.cat = cat
        self.index = index
        self.base_url = base_url
        request = requests.get(self.base_url.format(self.cat,self.index))
        
        soup = bs4.BeautifulSoup(request.text)
        
        return soup
        
    def Books_to_table(self,cat,Books_list):
        self.Books_list = Books_list
        #print(self.Books_list)
        self.cat = cat
        self.df = self.df.append({'Category':self.cat,'Names':self.Books_list}, ignore_index = True)
        return self.df

In [ ]:
Book = Book_Scrap()
df = pd.DataFrame()
start = time.time()

for cat in Books_list[1:]:
  
    url = Book.urls(base_url,cat,Books_list.index(cat)+1)
    Books_name = Book.books_name(url)
    df = Book.Books_to_table(cat,Books_name)
   
end_time = time.time()
elapsed_time = end_time - start
print("Total time to execute = ",elapsed_time)

In [ ]:
df = df.explode('Names')

In [ ]:
df.head()